In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

def getOlx():
    olx_list = []
    for i in tqdm(range(1,101), desc='OLX'):
        r = requests.get(f'https://www.olx.com.br/imoveis/estado-sp?o={i}', headers=headers)
        content = r.content
        soup = BeautifulSoup(r.content, 'html.parser') 
        li_tags = soup.find_all('li')
        # Iterando sobre as tags <li> encontradas
        for li_tag in li_tags:
            # Verificando se a tag <li> contém um link
            if li_tag.find('a'):
                # Obtendo o valor do atributo "href" do link
                link = li_tag.find('a')['href']        
                if link.startswith('https://sp.olx.com.br'):
                    olx_list.append(link)
    return olx_list

def getLoft():
    temp = []
    loft_list = []
    for i in tqdm(range(1, 133),desc='Loft'):
            response = requests.get(f'https://loft.com.br/venda/imoveis/sp/sao-paulo?utm_source=google&utm_medium=cpc&utm_campaign=ins_01_br_001_sp_0001_sao-paulo_all_aw_search_conversion_broad_&utm_content=all_loft-pura&utm_id=1756086660&utm_placement=&utm_ad_id=576438774862&gclid=Cj0KCQjwjryjBhD0ARIsAMLvnF9KvelK1iKF7xoTJ3HS0KSWMhoDO0UlANc1gTfQEMGg5Bud0HiPZg4aAscOEALw_wcB&pagina={i}')
            html_content = response.text

            soup = BeautifulSoup(html_content, 'html.parser')

            divs = soup.find_all('div', class_='MuiGrid-root')

            for div in divs:
                tag_a = div.find('a')
                if tag_a is not None:
                    link = tag_a['href']
                    if link.startswith('/imovel/'):
                        temp.append(link)
    loft_list = ['http://loft.com.br' + valor for valor in temp]
    return loft_list

## MAIN CODE
olx_list = getOlx()
loft_list = getLoft()

## get olx data


dados_gerais = []  # Lista para armazenar os dados de todas as URLs

for url in tqdm(olx_list):
    r = requests.get(url, headers=headers)
    content = r.content
    soup = BeautifulSoup(r.content, 'lxml') 

    h2 = soup.find('h2', class_='ad__sc-1wimjbb-1 hoHpcC sc-cooIXK cXlgiS')
    if h2:
        h2_value = h2.text.strip()
    else:
        h2_value = ''

    divs = soup.find_all('div', class_='sc-hmzhuo ad__sc-1f2ug0x-3 sSzeX sc-jTzLTM iwtnNi')  # Substitua 'nome_da_classe' pela classe real das divs

    dados_url = {}  # Dicionário para armazenar os dados de uma URL específica

    # Adicione a URL ao dicionário da URL específica com a chave 'URL'
    dados_url['URL'] = url

    # Adicione o valor da tag h2 ao dicionário da URL específica com a chave 'Valor'
    dados_url['Valor'] = h2_value

    # Itere sobre as divs encontradas
    for div in divs:
        # Inicialize um dicionário vazio para armazenar os valores da div atual
        dados = {}

        # Encontre todas as tags dt dentro da div atual
        tags_dt = div.find_all('dt')

        # Itere sobre as tags dt e obtenha os valores
        for dt in tags_dt:
            key = dt.text.strip()  # Use strip() para remover espaços em branco extras

            # Encontre a próxima tag dd
            dd = dt.find_next_sibling('dd')

            # Se a tag dd existir, obtenha o valor
            if dd:
                value = dd.text.strip()
            else:
                # Se não houver uma tag dd, encontre a próxima div com a classe específica
                proxima_div = div.find_next('div', class_='ad__sc-1f2ug0x-2 eSYIff')

                # Encontre a tag a dentro da próxima div
                a = proxima_div.find('a') if proxima_div else None

                # Obtenha o valor da tag a, se existir
                value = a.text.strip() if a else ''

            dados[key] = value

        # Adicione o dicionário da div atual ao dicionário da URL específica
        dados_url.update(dados)

    # Adicione o dicionário da URL específica à lista geral
    dados_gerais.append(dados_url)

# Converter a lista de dicionários em um DataFrame do pandas
df = pd.DataFrame(dados_gerais)

# Salvar o DataFrame em um arquivo Excel
df.to_excel('dados.xlsx', index=False)



Loft: 100%|██████████| 132/132 [00:27<00:00,  4.78it/s]
